In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import psutil


class VAE(nn.Module):
    def __init__(self, latent_dim=200):
        super(VAE, self).__init__()

        # Encoder: 4 Conv2d layers with stride=2, kernel=4, padding=1
        # Input: (3, 64, 64) -> (32, 32, 32) -> (64, 16, 16) -> (128, 8, 8) -> (256, 4, 4)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),    # 0
            nn.ReLU(),                                                # 1
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),   # 2
            nn.ReLU(),                                                # 3
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # 4
            nn.ReLU(),                                                # 5
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # 6
            nn.ReLU(),                                                # 7
        )

        # 256 * 4 * 4 = 4096
        self.fc_mu = nn.Linear(4096, latent_dim)
        self.fc_logvar = nn.Linear(4096, latent_dim)

        # Decoder input
        self.decoder_input = nn.Linear(latent_dim, 4096)

        # Decoder: 4 ConvTranspose2d layers
        # (256, 4, 4) -> (128, 8, 8) -> (64, 16, 16) -> (32, 32, 32) -> (3, 64, 64)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 0
            nn.ReLU(),                                                          # 1
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),   # 2
            nn.ReLU(),                                                          # 3
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),    # 4
            nn.ReLU(),                                                          # 5
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),     # 6
            nn.Sigmoid(),                                                       # 7
        )

    def encode(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten to (batch, 4096)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        x = self.decoder_input(z)
        x = x.view(x.size(0), 256, 4, 4)  # Reshape to (batch, 256, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from copy import deepcopy
from torch.optim import AdamW
import numpy as np
import rsa

# --- Classe Principale Greedy DDPM ---

class GreedyVAE:
    def __init__(self, model, device="cuda"):
        self.device = device
        self.model = model

        # Configuration par défaut
        self.config = {
            "layer_name": "decoder.6.weight",  # Couche cible
            "message": "VAE_WATERMARK",
             "watermark_size": 16,# Message à encoder
            "objective_size": (16, 16),     # (watermark_bits, weights_per_bit)
            "ratio": 0.5,                     # Ratio de pruning
            "threshold": 0.1,                 # Seuil pour la loss
            "lambda_wat": 0.01,               # Poids de la watermark loss
            "lr": 1e-4,
            "epochs": 30,
            "beta_kl":1.0
        }

        self.saved_keys = {}

    def _get_target_layer(self, model):
        """Récupère le tenseur des poids de la couche cible."""
        for name, param in model.named_parameters():
            if name == self.config["layer_name"]:
                return param
        raise ValueError(f"Paramètre {self.config['layer_name']} introuvable.")


    def keygen(self, message):
        """
        Génère un watermark aléatoire basé sur le message.
        Returns:
            watermark_sig: Watermark en {-1, +1}
            watermark: Watermark binaire en {0, 1}
        """
        watermark_size = self.config.get("watermark_size", 32)
        print(watermark_size)

        # Set seed based on message for reproducibility
        seed = hash(message) % (2**32)
        torch.manual_seed(seed)

        # Generate random binary watermark {0, 1}
        b_sig = torch.randint(0, 2, (watermark_size,)).tolist()

        # Convert to signs: 0 -> -1, 1 -> +1
        watermark_sig = torch.tensor(
            [-1.0 if i == 0 else 1.0 for i in b_sig],
            dtype=torch.float
        ).to(self.device)

        # Binary watermark {0, 1}
        watermark = torch.tensor(b_sig, dtype=torch.float32).to(self.device)

        return watermark_sig, watermark

    def weight_extraction(self, model):
        """
        Extrait et reshape les poids de la couche cible.
        Returns:
            Tensor de taille objective_size
        """
        target_param = self._get_target_layer(model)

        # Flatten les poids
        layer_weight = target_param.view(-1)

        # Tronquer pour être divisible par objective_size[1]
        obj_size = self.config['objective_size']
        total_needed = obj_size[0] * obj_size[1]

        if layer_weight.numel() < total_needed:
            raise ValueError(
                f"Layer has {layer_weight.numel()} weights, "
                f"but objective_size requires {total_needed}"
            )

        # Adaptive average pooling pour obtenir la bonne taille
        truncated = layer_weight[:layer_weight.numel() // obj_size[1] * obj_size[1]]
        pooled = F.adaptive_avg_pool1d(
            truncated[None, None],
            obj_size[0] * obj_size[1]
        ).squeeze(0).view(obj_size)

        return pooled



    def compute_residuals(self, weight_matrix):
        """Vectorized version - much faster."""
        num_rows, num_cols = weight_matrix.shape
        num_to_prune = int(num_cols * self.config['ratio'] + 0.5)

        weights_copy = weight_matrix.clone()

        # Vectorized pruning - no for loop!
        sorted_indices = torch.argsort(torch.abs(weights_copy), dim=1)

        # Create mask for weights to zero out
        mask = torch.ones_like(weights_copy, dtype=torch.bool)
        rows = torch.arange(num_rows, device=weights_copy.device).unsqueeze(1)
        mask[rows, sorted_indices[:, :num_to_prune]] = False

        # Apply mask
        weights_copy = weights_copy * mask

        # Mean of each row
        residual_vector = torch.mean(weights_copy, dim=1)

        return residual_vector



    @staticmethod
    def _compute_ber(extracted, target):
        """Calcule le Bit Error Rate."""
        return (extracted != target).float().mean().item()

    def embed(self, dataloader):
        """
        Incorpore le watermark Greedy pendant le finetuning.
        """
        print(f"--- Démarrage Embedding GREEDY ({self.config['layer_name']}) ---")
        print(f"Message: {self.config['message']}")
        print(f"Objective size: {self.config['objective_size']}")
        print(f"Pruning ratio: {self.config['ratio']}")

        watermarked_model = self.model
        watermarked_model.train()

        # 1. Génération des clés (watermark de 256 bits)
        watermark_sig, watermark = self.keygen(self.config["message"])

        # # Ajuster objective_size si nécessaire pour correspondre à 256 bits
        # if self.config['objective_size'][0] != 64:
        #     print(f"⚠️ Adjusting objective_size[0] to 256 (watermark size)")
        #     self.config['objective_size'] = (64, self.config['objective_size'][1])
        # Adjust objective_size to match watermark_size
        watermark_size = self.config.get("watermark_size",16)
        print(watermark_size)
        if self.config['objective_size'][0] != watermark_size:
            print(f"⚠️ Adjusting objective_size[0] to {watermark_size} (watermark size)")
            self.config['objective_size'] = (watermark_size, self.config['objective_size'][1])
        print(f"Watermark size: {len(watermark)} bits")

        # 2. Vérification initiale
        weight_extract = self.weight_extraction(watermarked_model)
        residual_vector = self.compute_residuals(weight_extract)
        print(f"Initial residual vector shape: {residual_vector.shape}")

        # 3. Optimiseur
        optimizer = torch.optim.AdamW(watermarked_model.parameters(), lr=self.config["lr"])
        mse_loss = nn.MSELoss()

        # 4. Boucle d'entraînement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{self.config['epochs']}")
            for batch_idx, (clean_images, _) in enumerate(pbar):
            # for clean_images, _ in pbar:
                clean_images = clean_images.to(self.device)

                optimizer.zero_grad()

                # B. Prédiction (Task Loss)
                recon, mu, logvar = watermarked_model(clean_images)

                # Reconstruction loss
                l_recon = F.mse_loss(recon, clean_images)

                # KL divergence
                l_kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

                # VAE total task loss
                l_main = l_recon + self.config["beta_kl"] * l_kl

                # C. Watermark Loss (Greedy regularization)
                weight_extract = self.weight_extraction(watermarked_model)
                residual_vector = self.compute_residuals(weight_extract)

                # Loss: encourage residual signs to match watermark signs
                # l_wat = ReLU(threshold - watermark_sig * residual_vector)
                l_wat = F.relu(
                    self.config['threshold'] -
                    (watermark_sig * residual_vector)
                ).sum()

                # Loss Totale
                l_total = l_main + self.config["lambda_wat"] * l_wat

                l_total.backward()
                optimizer.step()

                # D. Calcul du BER
                if batch_idx % 10 == 0:
                    ber, _ = self._extract_internal(watermarked_model, watermark)

                pbar.set_postfix(
                    L_Main=f"{l_main.item():.4f}",
                    L_Wat=f"{l_wat.item():.4f}",
                    BER=f"{ber:.4f}"
                )

                # Early stopping si BER = 0
                # if ber == 0.0 and l_wat.item() < 0.01:
                #     print("✅ Convergence atteinte!")
                #     break

            # if ber == 0.0:
            #     break

        # Sauvegarde des clés
        self.saved_keys = {
            "watermark": watermark,
            "watermark_sig": watermark_sig,
            "watermarked_model": watermarked_model
        }

        print(f"\n✅ Embedding terminé! BER final: {ber:.4f}")
        torch.save(self.saved_keys, "greedy_VAE_model_checkpoint.pt")
        return watermarked_model

    def _extract_internal(self, model, watermark):
        """
        Extraction interne (utilisée pendant l'embedding).
        """
        with torch.no_grad():
            weight_extract = self.weight_extraction(model)
            residual_vector = self.compute_residuals(weight_extract)

            # Prendre le signe des résidus
            signs = torch.sign(residual_vector)

            # Convertir -1 -> 0 pour obtenir le watermark binaire
            extracted_wm = signs.clone()
            extracted_wm[extracted_wm == -1] = 0

            ber = self._compute_ber(extracted_wm, watermark)

        return ber, extracted_wm

    def extract(self, model=None):
        """
        Extrait le watermark d'un modèle suspect.
        """
        if model is None:
            model = self.saved_keys["watermarked_model"]

        watermark = self.saved_keys["watermark"]

        # 1. Extraction des poids et calcul des résidus
        try:
            weight_extract = self.weight_extraction(model)
            residual_vector = self.compute_residuals(weight_extract)
        except ValueError as e:
            print(f"⚠️ Erreur lors de l'extraction: {e}")
            return 1.0, None

        # 2. Calcul du watermark extrait
        signs = torch.sign(residual_vector)
        extracted_wm = signs.clone()
        extracted_wm[extracted_wm == -1] = 0

        # 3. Calcul du BER
        ber = self._compute_ber(extracted_wm, watermark)

        print(f"BER Extrait: {ber:.4f}")
        return ber, extracted_wm


/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch
from torchvision import transforms

import os
import torch
import gc
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load from Hugging Face (no Google Drive issues)
print("Loading dataset...")


from datasets import load_from_disk
hf_dataset = load_from_disk("celeba_local")
# os.makedirs("./celeba_images/all", exist_ok=True)
# for i, item in enumerate(hf_dataset):
#     item['image'].save(f"./celeba_images/all/{i:06d}.jpg")
# del hf_dataset


class CelebAWrapper(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, transform):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]['image']
        if self.transform:
            image = self.transform(image)
        return image, 0

dataset = CelebAWrapper(hf_dataset, transform)
# del hf_dataset
# gc.collect()
# dataset = datasets.ImageFolder("./celeba_images", transform=transform)
print("Dataset loaded!")

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
print("loader loaded!")


Loading dataset...
Dataset loaded!
loader loaded!


In [4]:
# --- EXEMPLE D'EXÉCUTION ---
#load model
latent_dim = 200

# Initialize the model
model = VAE(latent_dim=latent_dim)

# Load the trained weights
model_path = "./vae_celeba_latent_200_epochs_10_batch_64_subset_80000.pth"
model.load_state_dict(torch.load(model_path))
model.to(device)
# 2. Embedding Greedy
greedy_defense = GreedyVAE(model, device=device)



watermarked_model = greedy_defense.embed(dataloader)

--- Démarrage Embedding GREEDY (decoder.6.weight) ---
Message: VAE_WATERMARK
Objective size: (16, 16)
Pruning ratio: 0.5
16
16
Watermark size: 16 bits
Initial residual vector shape: torch.Size([16])


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:27<00:00, 36.35it/s, BER=0.0000, L_Main=0.2948, L_Wat=0.0000]


✅ Embedding terminé! BER final: 0.0000


In [5]:
# 3. Test d'extraction sur le modèle watermarké
print("\n--- Test Extraction sur modèle watermarké ---")
ber, extracted_wm = greedy_defense.extract()




--- Test Extraction sur modèle watermarké ---
BER Extrait: 0.0000


In [6]:
# --- Fonction de Distillation (Attaque) ---

def run_distillation_attack_greedy(greedy_obj, dataloader, epochs=5, lr=1e-4):
    """
    Tente de transférer la fonctionnalité du modèle Greedy vers un modèle vierge.
    Vérifie si la marque (basée sur les résidus des poids) survit.
    """
    device = greedy_obj.device
    checkpoint = torch.load("greedy_VAE_model_checkpoint.pt", weights_only=False)

    # 1. Teacher (Gelé)
    teacher = checkpoint["watermarked_model"]
    teacher.eval()
    for p in teacher.parameters():
        p.requires_grad = False

    # 2. Student (Vierge - Même architecture)
    print("\n--- Initialisation du Student ---")
    student=VAE(latent_dim=latent_dim)
    student.load_state_dict(torch.load(model_path))
    student.to(device)
    student.train()

    # Sanity Checks
    teacher_ber, _ = greedy_obj.extract(teacher)
    student_ber, _ = greedy_obj.extract(student)
    print(f"[Check] BER Teacher: {teacher_ber:.4f}")
    print(f"[Check] BER Student (Avant): {student_ber:.4f}")

    optimizer = AdamW(student.parameters(), lr=lr)

    history = {"loss": [], "ber": []}

    print(f"\n--- Distillation Greedy ({epochs} epochs) ---")
    a=0
    for epoch in range(epochs):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0

        for clean_images, _ in pbar:
            clean_images = clean_images.to(device)


            # B. Distillation (Output Matching)
            with torch.no_grad():
                 target_pred,_,_ = teacher(clean_images)

            student_pred,_,_ = student(clean_images)

            loss = F.mse_loss(student_pred, target_pred)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(Loss=loss.item())

        # C. Vérification du BER après chaque epoch
        current_ber, _ = greedy_obj.extract(student)
        history["ber"].append(current_ber)
        history["loss"].append(running_loss / len(dataloader))

        print(f"👉 Fin Epoch {epoch+1} | Loss: {loss.item():.4f} | BER Student: {current_ber:.4f}")
        if current_ber==0.0 and a>=1:
            print("✅ Marque récupérée avec succès par distillation !")
            break
        elif current_ber==0.0 and a<1 :
            a+=1
        else:
            a=0

    return student, history

In [ ]:
# 4. Attaque par Distillation
student_model, attack_stats = run_distillation_attack_greedy(
    greedy_defense, 
    dataloader, 
    epochs=100)


--- Initialisation du Student ---
BER Extrait: 0.0000
BER Extrait: 0.4375
[Check] BER Teacher: 0.0000
[Check] BER Student (Avant): 0.4375

--- Distillation Greedy (100 epochs) ---


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:28<00:00, 35.70it/s, Loss=0.0083]


BER Extrait: 0.4375
👉 Fin Epoch 1 | Loss: 0.0083 | BER Student: 0.4375


Epoch 2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:26<00:00, 36.56it/s, Loss=0.0069]


BER Extrait: 0.4375
👉 Fin Epoch 2 | Loss: 0.0069 | BER Student: 0.4375


...

Epoch 13: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:26<00:00, 36.53it/s, Loss=0.00738]


BER Extrait: 0.5000
👉 Fin Epoch 13 | Loss: 0.0074 | BER Student: 0.5000


Epoch 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:29<00:00, 35.46it/s, Loss=0.00605]


BER Extrait: 0.5000
👉 Fin Epoch 14 | Loss: 0.0060 | BER Student: 0.5000


Epoch 15:  52%|████████████████████████████████████████████████████████████████████████████████▎                                                                          | 1640/3166 [00:45<00:42, 35.53it/s, Loss=0.0055]

In [ ]:
# 5. Visualisation des résultats
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# BER over epochs
axes[0].plot(attack_stats["ber"], marker='o')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('BER')
axes[0].set_title('BER Evolution During Distillation Attack')
axes[0].axhline(y=0.5, color='r', linestyle='--', label='Random guess (0.5)')
axes[0].legend()
axes[0].grid(True)

# Loss over epochs
axes[1].plot(attack_stats["loss"], marker='s', color='orange')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Distillation Loss Over Epochs')
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 6. Comparaison finale
print("\n" + "="*50)
print("RÉSUMÉ FINAL")
print("="*50)

teacher_ber, _ = greedy_defense.extract(greedy_defense.saved_keys["watermarked_unet"])
student_ber, _ = greedy_defense.extract(student_model)

print(f"\nBER Teacher (watermarké): {teacher_ber:.4f}")
print(f"BER Student (après distillation): {student_ber:.4f}")

